<a href="https://colab.research.google.com/github/Jofdiazdi/TalleresSimpy/blob/master/Talleres/1.%20Introduccion%20a%20simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial Simpy








 ## Conceptos basicos

Simpy es una libreria en python de simulacion discreta en donde el comportamiento de componentes activos como vehiculos, mensajes o clientes es modelado a partir de procesos. 

* **Proceso**: Un proceso e una funcion que modela el comprotamiento de una parte del modelo a estudiar. Los procesos "viven" en un ambiente y durante su ejecuccion pueden crear eventos e interactuar con otros procesos.

* **Ambiente**: Un ambiente es un entorno el cual nos proporciona la creacion y control de un reloj de simulacion y la capacidad de crear procesos y asociarlos al ambiente.

* **Evento**: Son sucesos instantaneos, es decir duracion 0 los cuales permiten la interaccion entre los procesos del ambiente. Un proceso puede se tratado como un evento Simpy.

Para poder usar simpy primero tenemos que isntalarlo usando este comando:


In [0]:
!pip install simpy #Instalamos simpy

Y es muy imortante importar al libreria antes de usarla

In [0]:
import simpy #Importamos simpy

 ### Procesos y el ambiente de ejecucion
 
Miremos el siguiente ejemplo del modelo de un carro el cual pasa por 2 eventos, parquear y arrancar:

In [2]:
def carro(env):
  while True :
    print('Empezo a parquear en el tiempo %d' % env.now)
    duracion_parqueo = 5
    yield env.timeout(duracion_parqueo)
    
    print('Arranco en el tiempo %d' % env.now)
    duracion_viaje = 2
    yield env.timeout(duracion_viaje)
    
env = simpy.Environment()
env.process(carro(env))
env.run(until=15)

Empezo a parquear en el tiempo 0
Arranco en el tiempo 5
Empezo a parquear en el tiempo 7
Arranco en el tiempo 12
Empezo a parquear en el tiempo 14


La funcion `carro(evn)` modela el comportamiento del carro, por lo tanto es un proceso.

Posemos distinguir 3 momentos importantes en el:
```
print('Empezo a parquear en el tiempo %d' % env.now) #1
duracion_parqueo = 5 #2
yield env.timeout(duracion_parqueo)#3

```

* La primera linea nos define un evento, muy sencillo, el cual nos avisa que el carro empezo a parquear. La declaracion `env.now` retorna el tiempo de reloj en ese instante de la simulacion.

* La segunda linea nos define el tiempo que va a durar el proceso de parqueo o mejor dicho cuanto va a durar el carro parqueado.

* La tercera linea mueve el reloj hacia el tiempo definido con la funcion `env.timeout()`

Con respecto a lo anterior, las sigueintes 3 lineas son muy similares en estructura, y nos modelan cuando el carro empieza a arrancar y cuanto dura en ese estado.

```
env = simpy.Environment()
env.process(carro(env))
env.run(until=15)

```

La funcion `Enviroment()` de simpy nos proporciona un ambiente de ejecusion donde se inicializara un reloj y se correran los procesos.

La funcion del `env.process(carro(env))` genera un proceso y lo asocia al ambiente creado.

Por ultimo la funcion `env.run(until=15)` simula el sistema por 15 unidades de tiempo

### Interaccion entre procesos

Los procesos en simpy pueden ser usados como eventos, en si son la libreria los maneja como un tipo de evento. Cuando lo esperas vuelves a esa parte de codigo cuando el proceso termina. 

Miremos esto en el siguiente ejemplo:

In [0]:
import simpy

def charge(env, duracion):
  yield env.timeout(duracion)
  
def carro(env):
  while True:
    print('Comienza a parquear y cargar en %d' % env.now)
    
    duracion_carga = 5
    
    yield env.process(charge(env,duracion_carga))
    
    print('Comienza a conducir en %d' % env.now)
    
    duracion_viaje = 2
    yield env.timeout(duracion_viaje)
    
env = simpy.Environment()
env.process(carro(env))
env.run(until=15)

Comienza a parquear y cargar en 0
Comienza a conducir en 5
Comienza a parquear y cargar en 7
Comienza a conducir en 12
Comienza a parquear y cargar en 14


## Referencias

Ejemplos y definiciones tomadas de la pagina oficial de simpy: https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html